# Class definition

In [231]:
import os
from pypdf import PdfReader
import fitz
import pandas as pd
import re
from abc import ABC, abstractmethod
import time
import json

In [ ]:
class PageIdentifier(ABC):
    def __init__(self, base_dir, result_dir, verbose=False):
        self.base_dir = base_dir
        self.result_dir = result_dir
        self.verbose = verbose
        self.pages = {
            "Aktiva": [],
            "Passiva": [],
            "GuV": []
        }
        self.runtime = 0
        
    def walk_dir(self):
        start_time = time.time()

        for root, _, files in os.walk(self.base_dir):
            for file in files:
                if file.endswith(".pdf"):
                    filepath = os.path.join(root, file)
                    if self.verbose:
                        print(filepath)
                    
                    try:
                        self.identify_pages(filepath)
                    except Exception as e:
                        print(f"Error reading {filepath}: {e}")

        end_time = time.time()
        self.runtime = end_time - start_time

    def __generate_dataframe(self):
        df_list = []
        for item in self.pages.keys():
            df = pd.DataFrame.from_records(self.pages[item], columns=['filepath', 'page'])
            df['type'] = item
            df_list.append(df)

        df = pd.concat(df_list, ignore_index=True)
        return(df)
    
    def __generate_benchmarking_dataframe(self):
        df = self.__generate_dataframe()

        df_truth = pd.read_csv("../benchmark_truth/aktiva_passiva_guv_table_pages_no_ocr.csv")
        df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type').reset_index(drop=True)

        df_common = df_truth.merge(df, on=['filepath', 'page', 'type'], how='inner')
        df_common

        df2 = df.merge(df_common[['filepath', 'page', 'type']], on=['filepath', 'page', 'type'], how='left', indicator=True)
        df2 = df2[df2['_merge'] == 'left_only'].drop(columns=['_merge'])

        df_temp = df_truth.join(
            df.set_index(['filepath', 'type']),
            on=['filepath', 'type'],
            how='left',
            rsuffix='_identified'
        )
        df_temp = df_temp[df_temp['page_identified'].isna()].drop(columns=['page_identified'])

        correct_identified = df_common
        wrong_identified = df2
        missing = df_temp

        df_list = [correct_identified, wrong_identified, missing, df_truth]
        for data in df_list:
            data['company'] = data['filepath'].str.split('/').str[2]
            data['file'] = data['filepath'].str.split('/').str[3]
            data = data.drop(columns=['filepath'])

        # df_truth = pd.read_csv("../benchmark_truth/aktiva_passiva_guv_table_pages_no_ocr.csv")
        # df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type').reset_index(drop=True)

        # df_common = df_truth.merge(df, on=['filepath', 'page', 'type'], how='inner')
        # df = df.merge(df_common[['filepath', 'page', 'type']], on=['filepath', 'page', 'type'], how='left', indicator=True)
        # df = df[df['_merge'] == 'left_only'].drop(columns=['_merge'])
        
        # df_temp = df_truth.join(
        #     df.set_index(['filepath', 'type']),
        #     on=['filepath', 'type'],
        #     how='left',
        #     rsuffix='_identified'
        # )

        # df_common['page_identified'] = df_common['page']
        # df_temp = pd.concat([df_temp, df_common], axis=0, ignore_index=True).reset_index(drop=True)

        # df_temp['company'] = df_temp['filepath'].str.split('/').str[2]
        # df_temp['file'] = df_temp['filepath'].str.split('/').str[3]
        # df_temp = df_temp.drop(columns=['filepath'])

        return df_list

    def save_to_csv(self):
        df = self.__generate_dataframe()
        df.to_csv(self.result_dir + "_identified_pages.csv", index=False)
        
    def get_dataframe(self):
        df = self.__generate_dataframe()
        return df

    def benchmark(self, verbose=True, save_to_json=True):
        df_list = self.__generate_benchmarking_dataframe()

        correct_identified = df_list[0]
        wrong_identified = df_list[1]
        missing = df_list[2]
        df_truth = df_list[3]
        
        if verbose:
            print(f"There are {df_truth.drop_duplicates(subset=['file', 'type', 'company', 'page']).shape[0]} unique tables.")
            print(f"Correctly identified: {len(correct_identified)}")
            print(f"Wrongly identified: {len(wrong_identified)}")
            print(f"Missing: {len(missing)}")
            print(f"Runtime: {self.runtime:.2f} seconds")

        results = {'correct': correct_identified, 'wrong': wrong_identified, 'missing': missing, 'runtime': self.runtime, 'special': [self.special_benchmark_information()]}

        if save_to_json:
            with open(f"{self.result_dir}.json", "w") as json_file:
                json.dump(
                    {key: value.to_json(orient='records') if hasattr(value, 'to_json') else value 
                    for key, value in results.items()}, 
                    json_file, 
                    indent=4, 
                    default=str
                )

        return results
        
    
    @abstractmethod
    def identify_pages(self, filepath):
        pass

    @abstractmethod
    def special_benchmark_information(self):
        pass

In [233]:
from pytesseract import image_to_string
from pdf2image import convert_from_path
from pdfminer.high_level import extract_text
from pdfminer.pdfpage import PDFPage
import pypdfium2 as pdfium
from docling_parse.pdf_parser import DoclingPdfParser, PdfDocument
from docling_core.types.doc.page import TextCellUnit
from pdfplumber import open as pdfplumber_open

class PageIdentifierRegex(PageIdentifier):
    def __init__(self, base_dir, result_dir, regex_patterns, verbose=False, ocr=False, pdfbackend='pymupdf'):
        super().__init__(base_dir, result_dir, verbose)
        self.regex_patterns = regex_patterns
        self.ocr = ocr
        self.pdfbackend = pdfbackend

    def __process_text(self, text, filepath, page_num):
        if not text.strip() and self.ocr:
            # If no text is found and OCR is enabled, perform OCR
            if self.verbose:
                print(f"Performing OCR on {filepath} page {page_num + 1}")
            images = convert_from_path(filepath, first_page=page_num + 1, last_page=page_num + 1)
            if images:
                text = image_to_string(images[0])

        for item in self.pages.keys():
            if all(
                re.search(term, text, re.IGNORECASE | re.S) if isinstance(term, str) else False
                for term in self.regex_patterns[item]
            ):
                self.pages[item].append((filepath, page_num + 1))
                if self.verbose:
                    print(f"Found {item} in {filepath} on page {page_num + 1}")
            
    def special_benchmark_information(self):
        return json.dumps(self.regex_patterns)

    def identify_pages(self, filepath):
        if self.pdfbackend == 'pypdf':
            reader = PdfReader(filepath)
            for page_num, page in enumerate(reader.pages):
                text = page.extract_text()
                self.__process_text(text, filepath, page_num)

        elif self.pdfbackend == 'pymupdf':
            doc = fitz.open(filepath)
            for page_num, page in enumerate(doc):
                text = page.get_text()
                self.__process_text(text, filepath, page_num)

        elif self.pdfbackend == 'pdfminer':
            with open(filepath, 'rb') as file:
                for page_num, page in enumerate(PDFPage.get_pages(file)):
                    text = extract_text(file, page_numbers=[page_num])
                    self.__process_text(text, filepath, page_num)

        elif self.pdfbackend == 'pdfium':
            pdf = pdfium.PdfDocument(filepath)
            for page_num in range(len(pdf)):
                page = pdf[page_num]
                text = page.get_textpage().get_text_range()
                self.__process_text(text, filepath, page_num)

        elif self.pdfbackend == 'docling-parse':
            parser = DoclingPdfParser()
            pdf_doc: PdfDocument = parser.load(
                path_or_stream="../Geschaeftsberichte/GESOBAU AG/GESOBAU_Geschaeftsbericht_2017.pdf"
            )

            # PdfDocument.iterate_pages() will automatically populate pages as they are yielded.
            for page_no, pred_page in pdf_doc.iterate_pages():
                text = '\n'.join([line.text for line in pred_page.iterate_cells(unit_type=TextCellUnit.LINE)])
                self.__process_text(text, filepath, page_no)

        elif self.pdfbackend == 'pdfplumber':
            with pdfplumber_open(filepath) as pdf:
                for page_num, page in enumerate(pdf.pages):
                    text = page.extract_text()
                    self.__process_text(text, filepath, page_num)

        else:
            raise ValueError(f"Unknown pdfbackend: {self.pdfbackend}")

            


# Benchmarking

## Simple Rexeg OCR

In [ ]:
simple_regex_patterns = {
    "Aktiva": [
        r"aktiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "Passiva": [
        r"passiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))"
    ]
}

page_identifier_regex_dummy = PageIdentifierRegex(
    base_dir = "../Geschaeftsberichte/degewo AG/",
    # base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_1",
    regex_patterns = simple_regex_patterns,
    verbose = False,
    ocr = True
)

# page_identifier_regex_dummy.walk_dir()

Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 1
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 2
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 3
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 4
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 5
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 6
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 7
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 8
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 9
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 10
Performing OCR on ../Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf page 11
Performing OCR on .

In [ ]:
ben_df_dummy = page_identifier_regex_dummy.benchmark()
ben_df_dummy['missing'][ben_df_dummy['missing']['company'] == 'degewo AG']

Correctly identified: 12
Wrongly identified: 48
Missing: 249


,page,type,page_identified,company,file
202,28,Aktiva,NaN,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...
203,29,Passiva,NaN,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...


In [188]:
images = convert_from_path("../Geschaeftsberichte/degewo AG/201231_degewo_Konzernlagebericht_Konzernabschluss.pdf", first_page=27 + 1, last_page=27 + 1)
if images:
    text = image_to_string(images[0])

text
re.search(r"((20\d{2}).*(20\d{2}))", text, re.IGNORECASE | re.S)

<re.Match object; span=(65, 1938), match='2020\n\nAKTIVA\n\nA. Anlagevermégen\nI. Immaterie>

## Simple Regex

In [114]:
simple_regex_patterns = {
    "Aktiva": [
        r"aktiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "Passiva": [
        r"passiva",
        r"((20\d{2}).*(20\d{2}))"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))"
    ]
}

page_identifier_regex_1_pypdf = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/pypdf_simple_regex",
    regex_patterns = simple_regex_patterns,
    verbose = False,
    pdfbackend='pypdf'
)

page_identifier_regex_1_pypdf.walk_dir()

In [ ]:
ben_df1_pypdf = page_identifier_regex_1_pypdf.benchmark()
# ben_df1_pypdf['missing']

There are 249 unique tables.
Correctly identified: 203
Wrongly identified: 672
Missing: 43
Runtime: 208.05 seconds


In [171]:
page_identifier_regex_1_pymupdf = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/pymupdf_simple_regex",
    regex_patterns = simple_regex_patterns,
    verbose = False,
    pdfbackend='pymupdf'
)

page_identifier_regex_1_pymupdf.walk_dir()

In [117]:
ben_df1_pymupdf = page_identifier_regex_1_pymupdf.benchmark()

There are 249 unique tables.
Correctly identified: 207
Wrongly identified: 711
Missing: 43
Runtime: 21.14 seconds


In [166]:
page_identifier_regex_1_pdfminer = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/pdfminer_simple_regex",
    regex_patterns = simple_regex_patterns,
    verbose = False,
    pdfbackend='pdfminer'
)

page_identifier_regex_1_pdfminer.walk_dir()

The PDF <_io.BufferedReader name='../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2014.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2014.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2014.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2014.pdf'> contains a metadata field indicating that it should not allow text extractio

In [179]:
ben_df1_pdfminer = page_identifier_regex_1_pdfminer.benchmark()

There are 249 unique tables.
Correctly identified: 201
Wrongly identified: 715
Missing: 44
Runtime: 821.65 seconds


In [241]:

page_identifier_regex_1_pdfium = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/pdfium_simple_regex",
    regex_patterns = simple_regex_patterns,
    verbose = False,
    pdfbackend='pdfium'
)

page_identifier_regex_1_pdfium.walk_dir()

/home/simon/anaconda3/envs/llm/lib/python3.13/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [178]:
ben_df1_pdfium = page_identifier_regex_1_pdfium.benchmark()

There are 249 unique tables.
Correctly identified: 206
Wrongly identified: 713
Missing: 43
Runtime: 11.79 seconds


In [197]:
page_identifier_regex_1_docling = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/docling_simple_regex",
    regex_patterns = simple_regex_patterns,
    verbose = False,
    pdfbackend='docling-parse'
)

page_identifier_regex_1_docling.walk_dir()

In [198]:
ben_df1_docling = page_identifier_regex_1_docling.benchmark()

There are 249 unique tables.
Correctly identified: 4
Wrongly identified: 1083
Missing: 0
Runtime: 1449.24 seconds


## Expended Regex

In [171]:
regex_patterns_2 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_2 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_2",
    regex_patterns = regex_patterns_2,
    verbose = False
)

page_identifier_regex_2.walk_dir()

In [ ]:
ben_df2 = page_identifier_regex_2.benchmark()
ben_df2['missing']

Correctly identified: 235
Wrongly identified: 805
Missing: 12


,page,type,page_identified,company,file
178,26,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
179,27,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
185,17,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2007_BBB.pdf
187,24,Aktiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
188,25,Passiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
189,27,GuV,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
190,26,Aktiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
191,27,Passiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
192,29,GuV,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
193,26,Aktiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...


## Exhaustive Regex

In [234]:
# Aktiva und Passiva in einigen AfStatBBB nicht korrekt formatiert
regex_patterns_3 = {
    "Aktiva": [
        r"a\s*k\s*t\s*i\s*v\s*a|a\s*k\s*t\s*i\s*v\s*s\s*e\s*i\s*t\s*e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "Passiva": [
        r"p\s*a\s*s\s*s\s*i\s*v\s*a|p\s*a\s*s\s*s\s*i\s*v\s*s\s*e\s*i\s*t\s*e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "GuV": [
        r"gewinn|guv",
        r"verlust|guv",
        r"rechnung|guv",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_3_pymupdf = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/pymupdf_exhaustive_regex",
    regex_patterns = regex_patterns_3,
    verbose = False,
    pdfbackend='pymupdf'
)

page_identifier_regex_3_pymupdf.walk_dir()

In [111]:
text = PdfReader("../Geschaeftsberichte/Amt für Statistik Berlin-Brandenburg/AP_Geschaeftsbericht_DE_2019_BBB.pdf").pages[32].extract_text()
re.search(r"p\s*a\s*s\s*s\s*i\s*v\s*a|p\s*a\s*s\s*s\s*i\s*v\s*s\s*e\s*i\s*t\s*e|eigenkapital", text, re.IGNORECASE | re.S)
# text

<re.Match object; span=(25, 32), match='PASSIVA'>

In [235]:
ben_df3_pymupdf = page_identifier_regex_3_pymupdf.benchmark()
ben_df3_pymupdf['missing']

There are 249 unique tables.
Correctly identified: 248
Wrongly identified: 1671
Missing: 10
Runtime: 22.09 seconds


,page,type,page_identified,company,file
1189,33,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2019_BBB.pdf
1194,9,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2021_BBB.pdf
1203,8,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2023_BBB.pdf
1211,18,Aktiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2011_BBB.pdf
1220,29,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
1225,9,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2022_BBB.pdf
1235,33,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
1239,27,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
1245,25,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2014_BBB.pdf
1253,18,GuV,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2007_BBB.pdf


In [242]:
# df = page_identifier_regex_3_pymupdf.get_dataframe()
df = page_identifier_regex_1_pdfium.get_dataframe()
df

,filepath,page,type
0,../Geschaeftsberichte/Berliner Bäder Betriebe/...,10,Aktiva
1,../Geschaeftsberichte/Berliner Bäder Betriebe/...,15,Aktiva
2,../Geschaeftsberichte/Berliner Bäder Betriebe/...,22,Aktiva
3,../Geschaeftsberichte/Berliner Bäder Betriebe/...,28,Aktiva
4,../Geschaeftsberichte/Berliner Bäder Betriebe/...,11,Aktiva
...,...,...,...
846,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,58,GuV
847,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,62,GuV
848,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,64,GuV
849,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,66,GuV


In [237]:
df_truth = pd.read_csv("../benchmark_truth/aktiva_passiva_guv_table_pages_no_ocr.csv")
df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type').reset_index(drop=True)
df_truth

,filepath,page,type
0,../Geschaeftsberichte/Berliner Bäder Betriebe/...,10,Aktiva
1,../Geschaeftsberichte/Berliner Bäder Betriebe/...,11,Passiva
2,../Geschaeftsberichte/Berliner Bäder Betriebe/...,13,GuV
3,../Geschaeftsberichte/Berliner Bäder Betriebe/...,28,Aktiva
4,../Geschaeftsberichte/Berliner Bäder Betriebe/...,29,Passiva
...,...,...,...
244,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,44,GuV
245,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,62,GuV
246,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,52,Aktiva
247,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,53,Passiva


In [243]:
df_common = df_truth.merge(df, on=['filepath', 'page', 'type'], how='inner')
df_common

,filepath,page,type
0,../Geschaeftsberichte/Berliner Bäder Betriebe/...,10,Aktiva
1,../Geschaeftsberichte/Berliner Bäder Betriebe/...,11,Passiva
2,../Geschaeftsberichte/Berliner Bäder Betriebe/...,13,GuV
3,../Geschaeftsberichte/Berliner Bäder Betriebe/...,28,Aktiva
4,../Geschaeftsberichte/Berliner Bäder Betriebe/...,29,Passiva
...,...,...,...
201,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,44,GuV
202,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,62,GuV
203,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,52,Aktiva
204,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,53,Passiva


In [244]:
df2 = df.merge(df_common[['filepath', 'page', 'type']], on=['filepath', 'page', 'type'], how='left', indicator=True)
df2 = df2[df2['_merge'] == 'left_only'].drop(columns=['_merge'])
df2

,filepath,page,type
1,../Geschaeftsberichte/Berliner Bäder Betriebe/...,15,Aktiva
2,../Geschaeftsberichte/Berliner Bäder Betriebe/...,22,Aktiva
4,../Geschaeftsberichte/Berliner Bäder Betriebe/...,11,Aktiva
6,../Geschaeftsberichte/Berliner Bäder Betriebe/...,15,Aktiva
7,../Geschaeftsberichte/Berliner Bäder Betriebe/...,22,Aktiva
...,...,...,...
846,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,58,GuV
847,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,62,GuV
848,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,64,GuV
849,../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch...,66,GuV


In [246]:
df_temp = df_truth.join(
    df.set_index(['filepath', 'type']),
    on=['filepath', 'type'],
    how='left',
    rsuffix='_identified'
)
df_temp = df_temp[df_temp['page_identified'].isna()].drop(columns=['page_identified'])
df_temp


,filepath,page,type
65,../Geschaeftsberichte/IBB/ibb_digitaler_gescha...,16,GuV
66,../Geschaeftsberichte/IBB/ibb_digitaler_gescha...,17,GuV
170,../Geschaeftsberichte/Amt für Statistik Berlin...,18,Aktiva
171,../Geschaeftsberichte/Amt für Statistik Berlin...,19,Passiva
173,../Geschaeftsberichte/Amt für Statistik Berlin...,28,Aktiva
174,../Geschaeftsberichte/Amt für Statistik Berlin...,29,Passiva
179,../Geschaeftsberichte/Amt für Statistik Berlin...,32,Aktiva
180,../Geschaeftsberichte/Amt für Statistik Berlin...,33,Passiva
185,../Geschaeftsberichte/Amt für Statistik Berlin...,24,Aktiva
186,../Geschaeftsberichte/Amt für Statistik Berlin...,25,Passiva


In [ ]:

df_temp = df_truth.join(
    df.set_index(['filepath', 'type']),
    on=['filepath', 'type'],
    how='left',
    rsuffix='_identified'
)

df_common['page_identified'] = df_common['page']
df_temp = pd.concat([df_temp, df_common], axis=0, ignore_index=True).reset_index(drop=True)

df_temp['company'] = df_temp['filepath'].str.split('/').str[2]
df_temp['file'] = df_temp['filepath'].str.split('/').str[3]
df_temp = df_temp.drop(columns=['filepath'])

In [120]:
page_identifier_regex_3_pypdf = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/pypdf_exhaustive_regex",
    regex_patterns = regex_patterns_3,
    verbose = False,
    pdfbackend='pypdf'
)

page_identifier_regex_3_pypdf.walk_dir()

In [121]:
ben_df3_pypdf = page_identifier_regex_3_pypdf.benchmark()
ben_df3_pypdf['missing']

There are 249 unique tables.
Correctly identified: 246
Wrongly identified: 1600
Missing: 8
Runtime: 219.63 seconds


,page,type,page_identified,company,file
380,16,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
381,17,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
1095,33,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2019_BBB.pdf
1099,9,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2021_BBB.pdf
1108,8,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2023_BBB.pdf
1131,9,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2022_BBB.pdf
1146,27,Passiva,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2009_BBB.pdf
1161,18,GuV,NaN,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2007_BBB.pdf


Findet nun alles, was kein OCR braucht, aber dafür auch doppelt so viel irrelevantes.

In [221]:
ben_df3['wrong'].groupby(['type', 'company']).agg(
    count=('page', 'count')
).sort_values('count', ascending=False)

count
type    company                                    
Passiva Berlinovo                               225
Aktiva  GESOBAU AG                              172
Passiva IBB                                     148
Aktiva  IBB                                     139
        Berlinovo                               129
        Berliner Bäder Betriebe                 117
GuV     Berlinovo                               105
Passiva GESOBAU AG                              105
GuV     Berliner Bäder Betriebe                  94
Passiva Berliner Bäder Betriebe                  83
GuV     IBB                                      80
        GESOBAU AG                               65
        Berlin Energie und Netzholding           40
        Amt für Statistik Berlin-Brandenburg     35
Aktiva  Berlin Energie und Netzholding           31
Passiva Berlin Energie und Netzholding           31
Aktiva  Amt für Statistik Berlin-Brandenburg     13
Passiva Amt für Statistik Berlin-Brandenburg      8
GuV     degewo AG                                 7
Passiva degewo AG                                 7
Aktiva  degewo AG                                 6

In [310]:
# still not correct identified tables
ben_df3['wrong'].drop(columns=['page_identified']).drop_duplicates().merge(
    ben_df3['correct'].drop(columns=['page_identified']).drop_duplicates(),
    on=['page', 'type', 'company', 'file'],
    how='left',
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge']).merge(
    ben_df3['missing'].drop(columns=['page_identified']).drop_duplicates(),
    on=['page', 'type', 'company', 'file'],
    how='left',
    indicator=True
).query('_merge == "left_only"').drop(columns=['_merge'])

,page,type,company,file
0,17,GuV,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
1,71,GuV,IBB,ibb_geschaeftsbericht_2006.pdf
2,24,Aktiva,degewo AG,degewo_Konzernlagebericht_Konzernabschluss_202...
3,25,Passiva,degewo AG,degewo_Konzernlagebericht_Konzernabschluss_202...
4,27,GuV,degewo AG,degewo_Konzernlagebericht_Konzernabschluss_202...
5,28,Aktiva,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...
6,29,Passiva,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...
7,31,GuV,degewo AG,201231_degewo_Konzernlagebericht_Konzernabschl...
8,43,GuV,Berlin Energie und Netzholding,SIGNATURE_DEE00084236.1.1_550912_Endexemplar_2...


## Exhaustive Regex OCR

In [ ]:
regex_patterns_3 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_3_ocr = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_3",
    regex_patterns = regex_patterns_3,
    verbose = True,
    ocr = True
)

# page_identifier_regex_3_ocr.walk_dir()

In [ ]:
ben_df3_ocr = page_identifier_regex_3_ocr.benchmark()
ben_df3_ocr['missing']

## Exhaustive Regex restricting

In [222]:
# Aktiva und Passiva in einigen AfStatBBB nicht korrekt formatiert
regex_patterns_4 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr",
        r"anlageverm.{1,2}gen"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr",
        r"eigenkapital"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr",
        r"umsatzerl.{1,2}se"
    ]
}

page_identifier_regex_4 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_3",
    regex_patterns = regex_patterns_4,
    verbose = False
)

page_identifier_regex_4.walk_dir()

Mehrseitige GuV in "ibb_geschaeftsbericht_2017.pdf"

In [223]:
ben_df4 = page_identifier_regex_4.benchmark()
ben_df4['missing']

Correctly identified: 184
Wrongly identified: 1121
Missing: 40


,page,type,page_identified,company,file
44,44,GuV,NaN,IBB,ibb_geschaeftsbericht_2017.pdf
47,6,GuV,NaN,IBB,ibb-jahresabschluss-und-lagebericht-2023.pdf
50,57,GuV,NaN,IBB,ibb_geschaeftsbericht_2020.pdf
53,100,GuV,NaN,IBB,ibb_geschaeftsbericht_2013.pdf
56,74,GuV,NaN,IBB,ibb_geschaeftsbericht_2014.pdf
59,71,GuV,NaN,IBB,ibb-geschaeftsbericht-2021.pdf
62,16,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
63,17,GuV,NaN,IBB,ibb_digitaler_geschaeftsbericht_2017.pdf
66,76,GuV,NaN,IBB,ibb_geschaeftsbericht_2007.pdf
69,74,GuV,NaN,IBB,ibb_geschaeftsbericht_2008.pdf


In [224]:
# Aktiva und Passiva in einigen AfStatBBB nicht korrekt formatiert
regex_patterns_5 = {
    "Aktiva": [
        r"aktiva|aktivseite|a k t i v a|a k t i v s e i t e|anlageverm.{1,2}gen",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr",
        r"anlageverm.{1,2}gen"
    ],
    "Passiva": [
        r"passiva|passivseite|p a s s i v a|p a s s i v s e i t e|eigenkapital",
        r"((20\d{2}).*(20\d{2}))|((20\d{2}).*vorjahr)|vorjahr",
        r"eigenkapital"
    ],
    "GuV": [
        r"gewinn",
        r"verlust",
        r"rechnung",
        r"((20\d{2}).*(20\d{2}))|vorjahr"
    ]
}

page_identifier_regex_5 = PageIdentifierRegex(
    # base_dir = "../Geschaeftsberichte/Berlin Energie und Netzholding/",
    base_dir = "../Geschaeftsberichte/",
    result_dir= "../benchmark_results/page_identification/regex_3",
    regex_patterns = regex_patterns_5,
    verbose = False
)

page_identifier_regex_5.walk_dir()

In [225]:
ben_df5 = page_identifier_regex_5.benchmark()
ben_df5['missing']

Correctly identified: 207
Wrongly identified: 1463
Missing: 11


,page,type,page_identified,company,file
187,24,Aktiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
188,25,Passiva,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
189,27,GuV,NaN,degewo AG,2017_degewo_degewo_Homepage.pdf
190,26,Aktiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
191,27,Passiva,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
192,29,GuV,NaN,degewo AG,2018_degewo_Konzernlagebericht_und_Konzernabsc...
193,26,Aktiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
194,27,Passiva,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
195,29,GuV,NaN,degewo AG,2019_degewo_Konzernbericht_und_Gleicher_Lohn_f...
196,24,Aktiva,NaN,degewo AG,degewo_Konzernlagebericht_Konzernabschluss_202...


# Tests

## Extracting text

In [301]:
text = PdfReader('../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2006.pdf').pages[67].extract_text()
text

'\x18\x18\nJahresbilanz zum 31. Dezember 2006\na ktivseite in te U r\n31.12.2006 31.12.2005\n1. Barreserve\n\t\n\t\nb)\t\n\t\nGuthaben\n\t\nbei\n\t\nZentralnotenbanken\t\ndarunter:\n\t\nbei\n\t\nder\n\t\nDeutschen\n\t\nBundesbank:\t\nTEUR\n\t\n19.823\n\t\n(31.12.2005\n\t\n:\n\t\nTEUR\n\t\n28.873)\n3. Forderungen an \nk\nreditinstitute\n\t a)\n\t\ntäglich\n\t\nfällig\n\t b)\n\t\nandere\n\t\nForderungen\n4. Forderungen an \nk\nunden\n\t\n\t\ndarunter:\n\t\t\ndurch\n\t\nGrundpfandrechte\n\t\ngesichert:\t\nTEUR\n\t\n9.496.661\n\t\n(31.12.2005\n\t\n:\n\t\nTEUR\n\t\n10.660.277)\t\nKommunalkredite:\n\t\nTEUR\n\t\n3.532.796\n\t\n(31.12.2005\n\t\n:\n\t\nTEUR\n\t\n2.338.961)\n5. Schuldverschreibungen und andere festverzinsliche Wertpapiere\n\t a)\n\t\n\tGeldmarktpapiere\n\t\n\t\t ab)\n\t\nvon\n\t\nanderen\n\t\nEmittenten\t\n\t b)\n\t\n\tAnleihen\n\t\nund\n\t\nSchuldverschreibungen\n\t\n\t\t ba)\n\t\nvon\n\t\nöffentlichen\n\t\nEmittenten\n\t\n\t\t darunter:\n\t\nbeleihbar\n\t\nbei\n\t\nder\n\t\nD

In [305]:
re.search(r"a\sktiva", text, re.IGNORECASE | re.S)

<re.Match object; span=(1380, 1387), match='a\nktiva'>

## OCR

In [231]:
file_list = []
for root, _, files in os.walk("../Geschaeftsberichte/"):
    for file in files:
        file_list.append(os.path.join(root, file))

file_list

missing_files = [file for file in file_list if file not in df_truth['filepath'].values]
missing_files

['../Geschaeftsberichte/degewo AG/240618-degewo-konzernlagerbericht-konzernabschluss-2023.pdf']

The is no table of interest in  "../Geschaeftsberichte/degewo AG/240618-degewo-konzernlagerbericht-konzernabschluss-2023.pdf"

In [ ]:
df_truth = pd.read_csv("../benchmark_truth/aktiva_passiva_guv_table_pages.csv")
df_truth = df_truth.assign(type=df_truth['type'].str.split('&')).explode('type')
df_truth.shape[0]

265

## Table detection

In [ ]:
from transformers import pipeline
from PIL import ImageDraw, ImageFont
# import warnings

font = ImageFont.truetype(font="/usr/share/fonts/truetype/ubuntu/Ubuntu-B.ttf", size = 20)
threshold = 0.95 # 48 Tabellen wurden nicht erkannt
threshold = 0.90 # 38 Tabellen wurden nicht erkannt
threshold = 0.70 # 22 Tabellen wurden nicht erkannt
threshold = 0.50 # 10 Tabellen wurden nicht erkannt

# warnings.filterwarnings("ignore")
pipe = pipeline("object-detection", model="microsoft/table-transformer-detection")

counter = {'with_table': 0, 'without_table': 0}

for _, row in ben_df1['correct'].iterrows():
    filepath = f"../Geschaeftsberichte/{row['company']}/{row['file']}"
    page_num = row['page']

    images = convert_from_path(filepath, first_page=page_num, last_page=page_num)
    results = pipe(images[0], threshold=threshold)

    if len(results) > 0:
        counter['with_table'] += 1
    else:
        counter['without_table'] += 1
        # Draw the bounding boxes on the image
        draw = ImageDraw.Draw(images[0])
        
        draw.text((10, 10), f"{filepath} at page: {page_num}", fill="red", font=font)

        # Show the image with the bounding boxes
        images[0].show()

print(counter)

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Created TensorFlow Lite XNNPACK delegate for CPU.
Attempting to use a delegate that only su

Opening in existing browser session.


[90078:90078:0509/181955.193433:ERROR:gl_surface_presentation_helper.cc(260)] GetVSyncParametersIfAvailable() failed for 2 times!


Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.


[90078:90078:0509/181958.465096:ERROR:gl_surface_presentation_helper.cc(260)] GetVSyncParametersIfAvailable() failed for 3 times!


Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
{'with_table': 181, 'without_table': 22}


In [ ]:
# ben_df1['wrong'].drop(columns=['page', 'type']).drop_duplicates().to_csv("../benchmark_results/page_identification/ben_df1_wrong.csv", index=False)

df_temp = ben_df1['wrong'].drop(columns=['page', 'type']).drop_duplicates()
df_temp['filepath'] = '../Geschaeftsberichte/' + df_temp['company'] + "/" + df_temp['file']

from pypdf import PdfWriter

writer = PdfWriter()
for _, row in df_temp.iterrows():
    filepath = row['filepath']
    page_num = int(row['page_identified']) - 1  # Page numbers are 0-indexed in PdfReader

    reader = PdfReader(filepath)
    writer.add_page(reader.pages[page_num])

with open("../benchmark_results/page_identification/ben_df1_wrong_pages.pdf", "wb") as output_pdf:
    writer.write(output_pdf)

In [ ]:
# ben_df1['wrong'].drop(columns=['page', 'type']).drop_duplicates().to_csv("../benchmark_results/page_identification/ben_df1_wrong.csv", index=False)

df_temp = ben_df1['correct'].drop(columns=['page', 'type']).drop_duplicates()
df_temp['filepath'] = '../Geschaeftsberichte/' + df_temp['company'] + "/" + df_temp['file']

from pypdf import PdfWriter

writer = PdfWriter()
for _, row in df_temp.iterrows():
    filepath = row['filepath']
    page_num = int(row['page_identified']) - 1  # Page numbers are 0-indexed in PdfReader

    reader = PdfReader(filepath)
    writer.add_page(reader.pages[page_num])

with open("../benchmark_results/page_identification/ben_df1_correct_pages.pdf", "wb") as output_pdf:
    writer.write(output_pdf)

In [ ]:
from transformers import pipeline
from PIL import ImageDraw
# import warnings

threshold = 0.95 #  Tabellen wurden nicht erkannt

# warnings.filterwarnings("ignore")
pipe = pipeline("object-detection", model="microsoft/table-transformer-detection")

counter = {'with_table': 0, 'without_table': 0}

verbose = True

for _, row in ben_df1['wrong'].iterrows():
    filepath = f"../Geschaeftsberichte/{row['company']}/{row['file']}"
    page_num = int(row['page_identified'])

    images = convert_from_path(filepath, first_page=page_num, last_page=page_num)
    results = pipe(images[0], threshold=threshold)
    
    if len(results) > 0:
        counter['with_table'] += 1

        if verbose:    
            # Draw the bounding boxes on the image
            print(ben_df1['wrong'].loc[_])
            print(f"Found {len(results)} tables in {filepath} on page {page_num}")
            draw = ImageDraw.Draw(images[0])
            for result in results:
                box = result['box']
                draw.rectangle([box['xmin'], box['ymin'], box['xmax'], box['ymax']], outline="red", width=3)
                draw.text((box['xmin']+10, box['ymin']+10), f"{filepath} at page: {page_num} ({result['score']:.2f})", fill="red", font=font)

                # Show the image with the bounding boxes
                images[0].show()
    else:
        counter['without_table'] += 1

print(counter)    

Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


page                                      10
type                                  Aktiva
page_identified                         22.0
company              Berliner Bäder Betriebe
file               GB_BBB_Infra_2022_low.pdf
Name: 1, dtype: object
Found 1 tables in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 22
page                                      13
type                                     GuV
page_identified                         35.0
company              Berliner Bäder Betriebe
file               GB_BBB_Infra_2022_low.pdf
Name: 6, dtype: object
Found 1 tables in ../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf on page 35
Opening in existing browser session.
page                                      28
type                                  Aktiva
page_identified                         22.0
company              Berliner Bäder Betriebe
file               GB_BBB_Infra_2022_low.pdf
Name: 10, dtype: object
Found 1 tables i

[92300:92300:0509/182149.573550:ERROR:gl_surface_presentation_helper.cc(260)] GetVSyncParametersIfAvailable() failed for 1 times!
[92300:92300:0509/182149.583856:ERROR:gl_surface_presentation_helper.cc(260)] GetVSyncParametersIfAvailable() failed for 2 times!


page                                           16
type                                       Aktiva
page_identified                              11.0
company                   Berliner Bäder Betriebe
file               2023_Geschaeftsbericht_BBB.pdf
Name: 18, dtype: object
Found 1 tables in ../Geschaeftsberichte/Berliner Bäder Betriebe/2023_Geschaeftsbericht_BBB.pdf on page 11
Opening in existing browser session.
page                                           17
type                                      Passiva
page_identified                              11.0
company                   Berliner Bäder Betriebe
file               2023_Geschaeftsbericht_BBB.pdf
Name: 19, dtype: object
Found 1 tables in ../Geschaeftsberichte/Berliner Bäder Betriebe/2023_Geschaeftsbericht_BBB.pdf on page 11
Opening in existing browser session.
page                                           19
type                                          GuV
page_identified                              21.0
company         

Created TensorFlow Lite XNNPACK delegate for CPU.
Attempting to use a delegate that only supports static-sized tensors with a graph that has dynamic-sized tensors (tensor#-1 is a dynamic-sized tensor).


...

page                                                   63
type                                              Passiva
page_identified                                      91.0
company                                        GESOBAU AG
file               GESOBAU_Gesch%C3%A4ftsbericht_2024.pdf
Name: 690, dtype: object
Found 1 tables in ../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch%C3%A4ftsbericht_2024.pdf on page 91
Opening in existing browser session.
page                                                   64
type                                                  GuV
page_identified                                      70.0
company                                        GESOBAU AG
file               GESOBAU_Gesch%C3%A4ftsbericht_2024.pdf
Name: 692, dtype: object
Found 1 tables in ../Geschaeftsberichte/GESOBAU AG/GESOBAU_Gesch%C3%A4ftsbericht_2024.pdf on page 70
Opening in existing browser session.
page                                                   64
type                                

[112598:112598:0100/000000.201376:ERROR:zygote_linux.cc(664)] write: Broken pipe (32)


page                                               54
type                                              GuV
page_identified                                  64.0
company                                    GESOBAU AG
file               GESOBAU_Geschaeftsbericht_2017.pdf
Name: 730, dtype: object
Found 1 tables in ../Geschaeftsberichte/GESOBAU AG/GESOBAU_Geschaeftsbericht_2017.pdf on page 64
Opening in existing browser session.


[112644:112644:0100/000000.764939:ERROR:zygote_linux.cc(664)] write: Broken pipe (32)


{'with_table': 310, 'without_table': 369}
